In [40]:
pip install matplotlib pandas sqlalchemy bokeh numpy matplotlib scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd

# Load the datasets
train_file = r"C:\Users\DELL\Downloads\MASTERSAssgn\Dataset2\train.csv"
test_file = r"C:\Users\DELL\Downloads\MASTERSAssgn\Dataset2\test.csv"
ideal_file = r"C:\Users\DELL\Downloads\MASTERSAssgn\Dataset2\ideal.csv"

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
ideal_functions = pd.read_csv(ideal_file)

# Strip any extra spaces in column names
train_data.columns = train_data.columns.str.strip()
test_data.columns = test_data.columns.str.strip()
ideal_functions.columns = ideal_functions.columns.str.strip()

# Step 2: Examine the column names and first few rows
print("Training Data - Columns and First Few Rows:")
print(train_data.columns)
print(train_data.head())

print("\nTest Data - Columns and First Few Rows:")
print(test_data.columns)
print(test_data.head())

print("\nIdeal Functions Data - Columns and First Few Rows:")
print(ideal_functions.columns)
print(ideal_functions.head())


Training Data - Columns and First Few Rows:
Index(['x', 'y1', 'y2', 'y3', 'y4'], dtype='object')
      x         y1         y2         y3        y4
0 -20.0  39.778572 -40.078590 -20.214268 -0.324914
1 -19.9  39.604813 -39.784000 -20.070950 -0.058820
2 -19.8  40.099070 -40.018845 -19.906782 -0.451830
3 -19.7  40.151100 -39.518402 -19.389118 -0.612044
4 -19.6  39.795662 -39.360065 -19.815890 -0.306076

Test Data - Columns and First Few Rows:
Index(['x', 'y'], dtype='object')
      x          y
0  17.5  34.161040
1   0.3   1.215102
2  -8.7 -16.843908
3 -19.2 -37.170870
4 -11.0 -20.263054

Ideal Functions Data - Columns and First Few Rows:
Index(['x', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11',
       'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19', 'y20', 'y21',
       'y22', 'y23', 'y24', 'y25', 'y26', 'y27', 'y28', 'y29', 'y30', 'y31',
       'y32', 'y33', 'y34', 'y35', 'y36', 'y37', 'y38', 'y39', 'y40', 'y41',
       'y42', 'y43', 'y44', 'y45', 'y46', 'y4

In [68]:
from sqlalchemy import create_engine, Table, Column, Integer, Float, String, MetaData
from sqlalchemy.exc import SQLAlchemyError

class DatabaseManager:
    def __init__(self, db_file):
        self.db_file = db_file
        self.engine = create_engine(f'sqlite:///{self.db_file}')
        self.metadata = MetaData()
    
    def create_tables(self):
        """Creates tables for training data, ideal functions, and test results."""
        try:
            # Table for training data (columns: x, y)
            self.training_table = Table('training_data', self.metadata,
                                        Column('id', Integer, primary_key=True),
                                        Column('x', Float),
                                        Column('y', Float))
            
            # Table for ideal functions (columns: x, y1, y2, ..., y50)
            self.ideal_functions_table = Table('ideal_functions', self.metadata,
                                               Column('id', Integer, primary_key=True),
                                               Column('x', Float))
            for i in range(1, 51):
                self.ideal_functions_table.append_column(Column(f'y{i}', Float))
            
            # Table for test results (columns: x, y, function, deviation)
            self.test_results_table = Table('test_results', self.metadata,
                                            Column('id', Integer, primary_key=True),
                                            Column('x', Float),
                                            Column('y', Float),
                                            Column('function', String),
                                            Column('deviation', Float))
            
            # Create all tables
            self.metadata.create_all(self.engine)
            print("Database tables created successfully.")
        except SQLAlchemyError as e:
            print(f"Error creating tables: {str(e)}")
    
    def insert_data(self, table, data):
        """Inserts data into the specified table."""
        try:
            with self.engine.connect() as conn:
                conn.execute(table.insert(), data)
            print("Data inserted successfully.")
        except SQLAlchemyError as e:
            print(f"Error inserting data: {str(e)}")

# Instantiate DatabaseManager and create tables
db_manager = DatabaseManager("masters_project.db")
db_manager.create_tables()


Database tables created successfully.


In [43]:
def insert_training_data(db_manager, train_data):
    # Strip any extra spaces in column names
    train_data.columns = train_data.columns.str.strip()

    # Prepare the data to insert into the database
    training_data = []
    
    # Loop through each row of the training data
    for _, row in train_data.iterrows():
        # Add each y1, y2, y3, y4 with their corresponding x-value
        for i in range(1, 5):  # y1, y2, y3, y4
            training_data.append({"x": row['x'], f"y{i}": row[f'y{i}']})

    # Insert the data into the database
    db_manager.insert_data(db_manager.training_table, training_data)


In [44]:
def insert_ideal_functions_data(db_manager, ideal_functions):
    # Strip any extra spaces in column names
    ideal_functions.columns = ideal_functions.columns.str.strip()

    # Prepare the data to insert into the database
    ideal_function_data = []
    
    # Loop through each row of the ideal functions
    for _, row in ideal_functions.iterrows():
        ideal_function_data.append({"x": row['x'], **{f"y{i}": row[f'y{i}'] for i in range(1, 51)}})

    # Insert the data into the database
    db_manager.insert_data(db_manager.ideal_functions_table, ideal_function_data)


In [45]:
def least_squares_error(self, data, ideal_function_column, y_column):
    total_error = 0
    for _, row in data.iterrows():
        x_value = row['x']  # Get the x value from the data
        actual_y_value = row[y_column]  # Get the actual y value for the current ideal function

        # Find the corresponding ideal y value based on x
        ideal_y_value = ideal_function_column.loc[ideal_function_column['x'] == x_value, y_column].values

        if len(ideal_y_value) == 0:
            print(f"Warning: x_value {x_value} not found in ideal function. Skipping this value.")
            continue

        total_error += (actual_y_value - ideal_y_value[0]) ** 2  # Calculate the squared error

    return total_error


In [46]:
def choose_best_ideal_function(self):
    chosen_functions = []

    # Iterate over the ideal functions (y1, y2, ..., y50)
    for i in range(1, 51):
        ideal_function_column = self.ideal_functions[['x', f'y{i}']]  # Get the x and y(i) columns

        # Calculate the least squares error for this ideal function based on test data
        error = self.least_squares_error(self.test_data, ideal_function_column, f'y{i}')
        chosen_functions.append((f'y{i}', error))  # Store the function label and its error

    # Sort by error and select the top 4 ideal functions with the least error
    chosen_functions.sort(key=lambda x: x[1])
    return chosen_functions[:4]


In [47]:
def insert_test_results(db_manager, test_data, chosen_functions):
    test_results = []
    
    # Loop through each test data point
    for _, row in test_data.iterrows():
        x_value = row['x']
        best_function = None
        best_deviation = float('inf')

        # Find the ideal function with the smallest deviation
        for func, _ in chosen_functions:
            ideal_y_value = db_manager.fetch_ideal_function_value(func, x_value)  # This is a helper function you will need to implement.
            actual_y_value = row[func]
            deviation = abs(actual_y_value - ideal_y_value)

            if deviation < best_deviation:
                best_deviation = deviation
                best_function = func
        
        # Store the test result with the best function and deviation
        test_results.append({"x": x_value, "y": row[best_function], "function": best_function, "deviation": best_deviation})

    # Insert the results into the database
    db_manager.insert_data(db_manager.test_results_table, test_results)


In [26]:
# Display the best 4 ideal functions and their errors
print("\nBest 4 Ideal Functions based on Least Squares Error:")
for func, err in best_ideal_functions:
    print(f"{func}: Error = {err}")



Best 4 Ideal Functions based on Least Squares Error:
y11: Error = 39863.734507262954
y9: Error = 41354.676560322
y2: Error = 41937.21139680377
y50: Error = 42000.62336956572


In [14]:
import unittest
from unittest.mock import patch
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

class IdealFunctionMapper:
    def __init__(self, train_data_path, ideal_data_path, test_data_path):
        self.train_data = self.load_and_clean_data(train_data_path)
        self.ideal_data = self.load_and_clean_data(ideal_data_path)
        self.test_data = self.load_and_clean_data(test_data_path)

    def load_and_clean_data(self, path):
        try:
            data = pd.read_csv(path)
            # Example cleaning (e.g., remove any NaN values)
            data = data.dropna()
            return data
        except Exception as e:
            print(f"Error loading or cleaning data from {path}: {e}")
            return pd.DataFrame()

    def map_test_data(self):
        # Logic for mapping test data to ideal function
        mapped_results = []
        for _, row in self.test_data.iterrows():
            x_value = row['x']
            closest_function = self.select_ideal_function(x_value)
            mapped_results.append({
                'x': x_value,
                'y': row['y'],
                'delta_y': row['y'] - self.train_data.loc[self.train_data['x'] == x_value, 'y'].values[0],
                'ideal_function': closest_function
            })
        return pd.DataFrame(mapped_results)

    def select_ideal_function(self, x_value):
        # Logic for selecting the ideal function based on x_value
        return 'y1'  # Example return value

    def save_to_database(self, data):
        try:
            # Example database setup
            engine = create_engine('sqlite:///:memory:')  # Use an in-memory database for testing
            data.to_sql('mapped_results', engine, if_exists='replace', index=False)
            Session = sessionmaker(bind=engine)
            session = Session()
            
            # Use the text() function for the SQL query
            result = session.execute(text('SELECT * FROM mapped_results')).fetchall()
            print("Database results:", result)  # Print the database results
            return result
        except Exception as e:
            print(f"Error saving to database: {e}")
            return []

    def bokeh_visualization(self, mapped_data):
        try:
            plot = figure(title="Mapped Data Visualization", x_axis_label='X', y_axis_label='Y')
            plot.scatter(mapped_data['x'], mapped_data['y'], legend_label="Mapped Data")
            return plot
        except Exception as e:
            print(f"Error in Bokeh visualization: {e}")
            return None

# Unit tests
class TestIdealFunctionMapper(unittest.TestCase):

    @patch('pandas.read_csv')
    def test_load_and_clean_data(self, mock_read_csv):
        # Prepare the mock data (e.g., a DataFrame)
        mock_data = pd.DataFrame({
            'x': [1, 2, 3],
            'y': [4, 5, 6]
        })
        mock_read_csv.return_value = mock_data
        
        # Now when the mapper tries to load the CSV, it gets the mocked data
        mapper = IdealFunctionMapper('mock_train.csv', 'mock_ideal.csv', 'mock_test.csv')
        
        # Ensure that the loaded data is what we mocked
        self.assertTrue(mapper.train_data.equals(mock_data))
        self.assertTrue(mapper.ideal_data.equals(mock_data))
        self.assertTrue(mapper.test_data.equals(mock_data))

    @patch('pandas.read_csv')
    def test_map_test_data(self, mock_read_csv):
        mock_data = pd.DataFrame({
            'x': [1, 2, 3],
            'y': [4, 5, 6]
        })
        mock_read_csv.return_value = mock_data
        
        # Create mapper
        mapper = IdealFunctionMapper('mock_train.csv', 'mock_ideal.csv', 'mock_test.csv')
        
        # Test mapping process
        mapped_data = mapper.map_test_data()
        self.assertIn('x', mapped_data.columns)
        self.assertIn('y', mapped_data.columns)
        self.assertIn('delta_y', mapped_data.columns)
        self.assertIn('ideal_function', mapped_data.columns)

    @patch('pandas.read_csv')
    def test_save_to_database(self, mock_read_csv):
        mock_data = pd.DataFrame({
            'x': [1, 2, 3],
            'y': [4, 5, 6]
        })
        mock_read_csv.return_value = mock_data
        
        mapper = IdealFunctionMapper('mock_train.csv', 'mock_ideal.csv', 'mock_test.csv')
        
        # Map data and save to database
        mapped_data = mapper.map_test_data()
        result = mapper.save_to_database(mapped_data)
        
        # Check if the result is not empty
        self.assertTrue(len(result) > 0)

    @patch('pandas.read_csv')
    def test_bokeh_visualization(self, mock_read_csv):
        mock_data = pd.DataFrame({
            'x': [1, 2, 3],
            'y': [4, 5, 6]
        })
        mock_read_csv.return_value = mock_data
        
        mapper = IdealFunctionMapper('mock_train.csv', 'mock_ideal.csv', 'mock_test.csv')
        
        # Map data and visualize with Bokeh
        mapped_data = mapper.map_test_data()

        # Initialize the notebook output
        output_notebook()  # Enables Bokeh plot output in Jupyter notebook
        plot = mapper.bokeh_visualization(mapped_data)
        
        # Check if the plot is an instance of Bokeh's figure
        self.assertIsInstance(plot, figure)
        
        # Display the plot
        show(plot)

if __name__ == '__main__':
    # Use TextTestRunner for better control over test execution in Jupyter
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestIdealFunctionMapper)
    runner = unittest.TextTestRunner()
    runner.run(suite)


Loading BokehJS ...

....
----------------------------------------------------------------------
Ran 4 tests in 0.348s

OK


Database results: [(1, 4, 0, 'y1'), (2, 5, 0, 'y1'), (3, 6, 0, 'y1')]
